# Метрики

## Imports

In [1]:
!pip install rectools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.4 MB/s 
     |████████████████████████████████| 1.1 MB 32.2 MB/s 
     |████████████████████████████████| 310 kB 58.4 MB/s 
     |████████████████████████████████| 13.5 MB 34.6 MB/s 
     |████████████████████████████████| 88 kB 8.1 MB/s 
     |████████████████████████████████| 60 kB 5.9 MB/s 
     |████████████████████████████████| 188 kB 63.3 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3411028 sha256=dfdc60dd33d67f6086a0c135131b515e8793c6184fb88cae20f0607c1690aa1e
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705362 sha256=c6b0f37cd239cfde6052b22fc072e7d2c5778e440aa14e55e2c34f636f9a1279
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3b

In [2]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

In [3]:
req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

NameError: ignored

In [ ]:
import zipfile as zf

files = zf.ZipFile('kion.zip','r')
files.extractall()
files.close()

In [ ]:
import pandas as pd
import numpy as np
import numba as nb

from tqdm.auto import tqdm
from rectools import Columns

In [ ]:
np.random.seed(23)

## Read data

In [ ]:
interactions = pd.read_csv('data_original/interactions.csv')

interactions.rename(
    columns={
        'track_id': Columns.Item,
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    }, 
    inplace=True) 

interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime])

In [ ]:
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

In [ ]:
def headtail(df):
    return pd.concat([df.head(), df.tail()])

headtail(interactions)

In [ ]:
interactions.info(memory_usage='deep')

Выделим небольшой кусок из данных, чтобы не слишком страдать

In [ ]:
sample_users = [57607, 403227, 70720]
df = interactions[interactions[Columns.User].isin(sample_users)].reset_index(drop=True)
del df[Columns.Datetime], df[Columns.Weight], df['watched_pct']
df

In [ ]:
print('Users', df[Columns.User].unique())
sample_items = df[Columns.Item].unique()
print('Items', sample_items)

## Regression

В регрессией все относительно просто. По (user, item) мы знаем таргет (рейтинг чаще всего) и по такой же паре предсказываем его

In [ ]:
df['target'] = np.random.choice([3, 4, 5], df.shape[0])
df['predict'] = np.random.rand(df.shape[0]) * 3 + 2
df

Общая оценка

In [ ]:
mae = (df['target'] - df['predict']).abs().mean()
print(mae)

Оценка по пользователю с последюущим усреднением

In [ ]:
df['diff'] = (df['target'] - df['predict']).abs()
average_mae = df.groupby(Columns.User)['diff'].mean()
print(average_mae.mean())
average_mae

Видно, что в данном случае метрики близки к друг другу, но это не всегда так

In [ ]:
del df['target'], df['predict'], df['diff']

## Classification

Сгенерируем случайные рекомендации.

In [ ]:
top_k = 5
recs = np.array([
    np.random.choice(sample_items, top_k, replace=False),
    np.random.choice(sample_items, top_k, replace=False),
    np.random.choice(sample_items, top_k, replace=False),
])
recs

Преобразуем в длинный датафрейм

In [ ]:
df_recs = pd.DataFrame({
    Columns.User: np.repeat(sample_users, top_k),
    Columns.Item: recs.ravel()
})
df_recs

In [ ]:
df_recs[Columns.Rank] = df_recs.groupby(Columns.User).cumcount() + 1
headtail(df_recs)

Ключевой момент. Именно ради него преобразовывали данные и именно это позволяет считать метрики быстрее.

In [ ]:
df_recs = df.merge(df_recs, how='left', left_on=Columns.UserItem, right_on=Columns.UserItem)
df_recs = df_recs.sort_values(by=[Columns.User, Columns.Rank])
df_recs

### Precision@K

In [ ]:
df_recs[f'TP@5'] = df_recs['rank'] < 6
df_recs

In [ ]:
df_recs[df_recs[Columns.Rank].notnull()]

Посчитаем вручную (1/5 + 1/5 + 3/5) / 3

In [ ]:
df_recs['TP@5/5'] = df_recs['TP@5'] / top_k 

p5 = df_recs.groupby(Columns.User)['TP@5/5'].sum().mean()

print(f'Precision@5 = {p5}')

In [ ]:
df_recs

Используем тот факт, что мы знаем количество пользователей, а значит groupby не нужен

In [ ]:
p5 = df_recs['TP@5/5'].sum() / len(sample_users)
print(f'Precision@5 = {p5}')

### Recall@K

In [ ]:
df_recs['actual'] = df_recs.groupby(Columns.User)[Columns.Item].transform('count')
df_recs

In [ ]:
df_recs['TP@5/actual'] = df_recs['TP@5'] / df_recs['actual']
df_recs

In [ ]:
(1/3 + 1/3 + 3/4) / 3

In [ ]:
r5 = df_recs.groupby(Columns.User)['TP@5/actual'].sum().mean()
print(f'Recall@5 = {r5}')

In [ ]:
r5 = df_recs['TP@5/actual'].sum() / len(sample_users)
print(f'Recall@5 = {r5}')

## Ranking

### MAP@K

In [ ]:
df_recs

In [ ]:
df_recs['cumTP@5'] = df_recs.groupby(Columns.User)['TP@5'].cumsum()
df_recs

In [ ]:
df_recs['Prec@5'] = df_recs['cumTP@5'] / df_recs[Columns.Rank]
df_recs

In [ ]:
df_recs['Prec@5/actual'] = df_recs['Prec@5'] / df_recs['actual']
df_recs

In [ ]:
ap = df_recs.groupby(Columns.User)['Prec@5/actual'].sum()
print(ap.mean())
ap

## Naive vs Numba vs Pandas

In [ ]:
df

In [ ]:
target = df.values
target

In [ ]:
target[target[:, 0] == 513902][:, 1]

In [ ]:
recs

In [ ]:
def precision_naive(target, users, recs, k):
    precision = []
    for i, user in enumerate(users):
        p = 0
        user_target = target[target[:, 0] == user][:, 1]
        for rec in recs[i]:
            if rec in user_target:
                p += 1
        precision.append(p / k)
    return sum(precision) / len(users)

In [ ]:
precision_naive(target, sample_users, recs, 5)

In [ ]:
@nb.njit(cache=True, parallel=True)
def precision_numba(target, users, recs, k):
    precision = np.zeros(len(users))
    for i in nb.prange(len(users)):
        user = users[i]
        p = 0
        user_target = target[target[:, 0] == user][:, 1]
        for rec in recs[i]:
            if rec in user_target:
                p += 1
        precision[i] = p / k
    return precision.mean()

In [ ]:
precision_numba(target, np.array(sample_users), recs, 5)

In [ ]:
precision_numba(target, np.array(sample_users), recs, 5)

In [ ]:
def precision_pandas(df, users, recs, k):
    df_recs = pd.DataFrame({
        Columns.User: np.repeat(users, k),
        Columns.Item: recs.ravel()
    })
    df_recs[Columns.Rank] = df_recs.groupby(Columns.User).cumcount() + 1
    df_recs = df.merge(df_recs, how='left', left_on=Columns.UserItem, right_on=Columns.UserItem)
    tp_k = f'TP@{k}'
    df_recs[tp_k] = df_recs[Columns.Rank] < (k + 1)
    p = df_recs[tp_k].sum() / k / len(users)
    return p

In [ ]:
precision_pandas(df, sample_users, recs, 5)

Посмотрим через `timeit`

In [ ]:
%timeit precision_naive(target, sample_users, recs, 5)

In [ ]:
%timeit precision_numba(target, sample_users, recs, 5)

In [ ]:
%timeit precision_pandas(df, sample_users, recs, 5)

In [ ]:
def generate_subsample(users_count, top_k):
    users = np.random.choice(interactions[Columns.User].unique(), users_count, replace=False)
    df = interactions[interactions[Columns.User].isin(users)].reset_index(drop=True)
    del df[Columns.Datetime], df[Columns.Weight], df['watched_pct']
    
    recs = np.random.choice(df[Columns.Item], size=(users_count, top_k))
    return df, users, recs

In [ ]:
top_k = 10
df, users, recs = generate_subsample(10000, top_k)
target = df.values

In [ ]:
%timeit precision_naive(target, users, recs, top_k)

In [ ]:
precision_numba(target, users, recs, top_k)

In [ ]:
%timeit precision_numba(target, users, recs, top_k)

In [ ]:
%timeit precision_pandas(df, users, recs, top_k)

## RecTools

Рассмотрим, как использовать библиотеку от МТС для подсчета метрик.

Полный гайд тут - [RecTools/examples/3_metrics.ipynb](https://github.com/MobileTeleSystems/RecTools/blob/main/examples/3_metrics.ipynb)

Вначале вспомним, какие данные есть у нас

In [ ]:
df.shape, users.shape, recs.shape

In [ ]:
from rectools.metrics import Precision, Recall, MAP, calc_metrics

In [ ]:
metrics = {
    "prec@1": Precision(k=1),
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "MAP@5": MAP(k=5),
    "MAP@10": MAP(k=10),
}

In [ ]:
catalog = df[Columns.Item].unique()

In [ ]:
df_recs = pd.DataFrame({
    Columns.User: np.repeat(users, top_k),
    Columns.Item: recs.ravel()
})
df_recs[Columns.Rank] = df_recs.groupby(Columns.User).cumcount() + 1

In [ ]:
metric_values = calc_metrics(
    metrics,
    reco=df_recs,
    interactions=df,
)

In [ ]:
metric_values

Как посчитать одну метрику

In [ ]:
metrics['prec@10'].calc(df_recs, df)

In [ ]:
%timeit metrics['prec@10'].calc(df_recs, df)

In [ ]:
metrics['prec@10'].calc_per_user(df_recs, df)

## Homework

### PFound
Исходные данные - Yandex Cup 2022 Analytics
- Ссылка - https://yandex.ru/cup/analytics/analysis/ , пример A. Рассчитать pFound
- Данные - https://yadi.sk/d/guqki4UI4hFlXQ
- Формула
$$pFound@K = \sum_{i=1}^{k} pLook[i]\ pRel[i]$$

$$pLook[1] = 1$$

$$pLook[i] = pLook[i-1]\ (1 - pRel[i-1])\ (1 - pBreak)$$

$$pBreak = 0.15$$

**Задача** - написать функцию, которая принимает на вход dataframe (после join), а на выходе дает средний pFound по всем query.
- Запрещается использовать циклы for для расчет метрики (как полностью, так и ее частей).
- Усложнение, если задача показалась легкой - попробуйте обойтись без groupby (не уверен, что это возможно, но вдруг вы справитесь)

In [5]:
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://yadi.sk/d/guqki4UI4hFlXQ'


final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

download_response = requests.get(download_url)
with open('./data.zip', 'wb') as f:
    f.write(download_response.content)

In [7]:
# !unzip data.zip
# !unzip hidden_task.zip
# !unzip open_task.zip

Archive:  hidden_task.zip
  inflating: hostid_url.tsv          
  inflating: qid_query.tsv           
  inflating: qid_url_rating.tsv      
Archive:  open_task.zip
   creating: open_task/
  inflating: open_task/qid_query.tsv  
  inflating: open_task/hostid_url.tsv  
  inflating: open_task/qid_url_rating.tsv  


##Pfound

In [18]:
qid_query.head(3) #  id запроса и текст запроса, разделённые табуляцией;

,qid,query
0,402111,работа фотографом в австралии
1,405851,производительность видеокарт
2,407522,ёлочные игрушки из пластиковых бутылок


In [13]:
qid_url_rating.head(3) #  id запроса, URL документа, релевантность документа запросу;

,qid,url,rating
0,402111,http://24-job.com/board/job_australia/232-1-2-...,0.07
1,402111,http://24-job.com/board/job_australia/232-1-2-...,0.07
2,402111,http://802351.info/5964-v-avstralii.html,0.00


In [15]:
 hostid_url.head(3) #  id хоста и URL документа.

,hostid,url
0,1,http://09spravki.ru/requisites.php
1,10,http://3pu.info/seo-tools/domains
2,1006,http://www.priroda.su/item/820


In [19]:
qid_url_rating_hostid.sample(3) 

,qid,url,rating,hostid
583,70357,http://necessary-soft.net/misc/103661-telefonn...,0.14,373
773,99543,http://spb.kp.ru/2007/02/01/doc161469,0.00,521
390,58989,http://www.mp3zzz.ru/song/27511.html,0.00,945


In [ ]:
import pandas as pd

# считываем данные
qid_query = pd.read_csv("qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("hostid_url.tsv", sep="\t", names=["hostid", "url"])

# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")


def plook(ind, rels):
    if ind == 0:
        return 1
    return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)


def pfound(group):
    max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста
    top10 = max_by_host.sort_values(ascending=False)[:10] # берем топ10 урлов с наивысшим рейтингом
    pfound = 0
    for ind, val in enumerate(top10):
        pfound += val*plook(ind, top10.values)
        print(plook(ind, top10.values))
    return pfound


qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # группируем по qid и вычисляем pfound
qid_max = qid_pfound.idxmax() # берем qid с максимальным pfound

qid_query[qid_query["qid"] == qid_max]


In [21]:
avg_pfound_original = qid_pfound.mean()
print(f'Средний pFound по всем query (расчет с сайта): {avg_pfound_original}')

Средний pFound по всем query (расчет с сайта): 0.5822199638393889


In [63]:
def fast_PFoundK(
    qid_url_rating_hostid,
    K = 10,
    p_break = 0.15
    ):
    """
    Подсчет метрики PFound@K
    """
    max_by_host = qid_url_rating_hostid.groupby(['qid', 'hostid'])['rating'].max().reset_index()
    top = max_by_host.sort_values(['qid', 'rating'], ascending=False).groupby(['qid']).head(10)

    top['rank'] = top.groupby('qid').cumcount()
    top['one_minus_rating'] = (1 - top['rating']).shift(1)
    top['p_break'] = 1 - p_break
    top.loc[top['rank'] == 0, ['p_break', 'one_minus_rating']] = 1

    top['temp_with_mult'] = top['one_minus_rating'] * top['p_break']

    top['plook'] = top.groupby('qid')['temp_with_mult'].cumprod()
    top['pfound'] = top['plook'] * top['rating']
    
    pfound_qid = top.groupby('qid')['pfound'].sum()
    return pfound_qid.mean()

In [64]:
print(fast_PFoundK(qid_url_rating_hostid))

0.5822199638393889


##MRR

### MRR
Исходные данные - результат `generate_subsample` 

**Задача** - по аналогии с precision написать три версии функции подсчета Mean Reciprocal Rank (naive, numba, pandas) и протестировать на разных размерах выборки
- Протестируйте для всех комбинаций (users_count, top_k):
  - users_count - [100, 1000, 10000, 100000]
  - top_k - [10, 50, 100]
- Результатом тестирования должен быть график, где будут отражены следующие показатели:
  - Алгоритм - naive, numba, pandas
  - Скорость работы (время)
  - users_count
  - top_k